# Simple  Tidy3D performance test

In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from scipy.io import loadmat

import tidy3d as td
from tidy3d import web
from tidy3d import SpatialDataArray
import warnings
td.config.logging_level = 'ERROR'

In [2]:
current_abs_path = os.path.abspath('')
sys.path.append(current_abs_path)
from adjoint import *

## Inverse design (FDTD)

In [3]:
interval = 0 # waiting time for each simulation
nm = 1e-3
n_out = 1.6380 # photoresist
wl0 = 355*nm
freq0 = td.C_0 / wl0
wl = wl0 / n_out
k0 = 2*np.pi/wl
period = 1000*nm
dkx = 2*np.pi/period
impedance = 377 / n_out
thickness = 150*nm
n_max = 2.9734 + 0.0467j  # TiO2
n_min = 1.4338  # PDMS
num_params = 100
num_iters = 100

gradient = np.zeros(num_params)
step_size = 0.2
target = {-3:0.0, -2:0.0, -1:0.15, 0:0.0, 1: 0.63, 2:0.0, 3:0.22}
angles = calculate_angles(k0, period, target)
gridx = np.linspace(-500, 500, num_params + 1) * nm
gridy = [0] #np.linspace(0, thickness, 16)
gridz = [0] #np.linspace(-5, 5, 2) * nm

foms = np.zeros(num_iters)
rho_arr = np.zeros(shape=(num_iters, num_params))
gradient_arr = np.zeros(shape=(num_iters, num_params))

In [4]:
# material
rho = 0.5 + 0.1*(np.genfromtxt(f"{current_abs_path}/grating_random_seed.csv",delimiter=",")-0.5)
mask_structure = td.Structure(geometry=td.Box(center=(0,90*nm, 0),size=(td.inf,180*nm,10*nm)),
                             medium = generate_nk(rho, n_max, n_min, gridx, gridy, gridz),
                             name="mask")
photoresist_structure = td.Structure(geometry=td.Box(center=(0, -1000*nm,0),size=(2000*nm,2000*nm,100*nm)),
                                 medium = td.Medium.from_nk(n_out,0,freq0),
                                 name = "photoresist")

In [5]:
# monitor
design_monitor = td.FieldMonitor(
    center=(0, 90*nm, 0), size=(2000*nm, 180*nm, 0), freqs=[freq0], name="design monitor"
)
field_monitor= td.FieldMonitor(
    center=(0, -500*nm, 0), size=(2500*nm, 0, 0), freqs=[freq0], name="field monitor"
)

In [6]:
sim_params=dict(
    center=(0, 0, 0),
    size=(1000*nm,1600*nm,0),
    medium=td.Medium.from_nk(n_min,0,freq0),
    grid_spec=td.GridSpec.uniform(dl=10*nm),
    structures=[photoresist_structure, mask_structure],
    monitors=[design_monitor, field_monitor],
    boundary_spec=td.BoundarySpec(
        x=td.Boundary(plus=td.Periodic(),minus=td.Periodic()),
        y=td.Boundary(plus=td.PML(),minus=td.PML()),
        z=td.Boundary(plus=td.Periodic(),minus=td.Periodic())
    ),
    run_time=2e-11,  # simulation run time,
    shutoff=1e-4,
    subpixel=False,
    normalize_index=None
)

In [7]:
# get normalized value
sim_params['sources'] = [td.PlaneWave(
    center=(0, 500*nm, 0),
    direction="-",
    size=(td.inf, 0, td.inf),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 20, amplitude = 1, phase = 0),
    pol_angle=np.pi / 2,
)]
sim_params['structures']=[]
sim_params['medium']=td.Medium.from_nk(n_out,0,freq0)
    
sim = td.Simulation(**sim_params)
if os.path.isfile(f"{current_abs_path}/data/test.hdf5"):
    sim_data = td.SimulationData.from_file(fname=f"{current_abs_path}/data/test.hdf5")
else:
    sim_data = web.run(
            sim,
            task_name=f"test",
            path=f"{current_abs_path}/data/test.hdf5",
            verbose=False
        )
Efield_norm_value = float(sim_data.load_field_monitor('design monitor').Ez.abs.mean())

In [11]:
sim_params['medium']=td.Medium.from_nk(n_min,0,freq0)
n = 'just_for_validation'
mask_structure = td.Structure(geometry=td.Box(center=(0,90*nm, 0),size=(td.inf,180*nm,10*nm)),
                             medium = generate_nk(rho, n_max, n_min, gridx, gridy, gridz),
                             name="mask")
sim_params['structures']=[photoresist_structure, mask_structure]
fom, E, H, E_fwd, factors, t_conj = forward_sim(sim_params, n, target, wl, period, impedance, freq0, Efield_norm_value)
E_adj = adjoint_sim(sim_params, n, factors, t_conj, angles, freq0, Efield_norm_value)

Diffraction intensity = [2.29356636e-04 4.17739348e-04 3.05530510e-04 8.74687002e-01
 2.78754611e-05 4.87737373e-04 2.83611956e-04]
target = [0.   0.   0.15 0.   0.63 0.   0.22]


In [9]:
sim_data = td.SimulationData.from_file(f'{current_abs_path}/data/forward_just_for_validation.hdf5')
print(sim_data.__dict__['log']) # see 'Time-stepping time (s)'

[05:40:12] USER: Simulation domain Nx, Ny, Nz: [100, 184, 1]                    
           USER: Applied symmetries: (0, 0, 0)                                  
           USER: Number of computational grid points: 1.8600e+04.               
           USER: Subpixel averaging method: SubpixelSpec(attrs={},              
           dielectric=Staircasing(attrs={}, type='Staircasing'),                
           metal=Staircasing(attrs={}, type='Staircasing'),                     
           pec=Staircasing(attrs={}, type='Staircasing'), type='SubpixelSpec')  
           USER: Number of time steps: 8.5651e+05                               
           USER: Automatic shutoff factor: 1.00e-04                             
           USER: Time step (s): 2.3351e-17                                      
           USER:                                                                
                                                                                
           USER: Compute sou

In [10]:
sim_data = td.SimulationData.from_file(f'{current_abs_path}/data/adjoint_just_for_validation.hdf5')
print(sim_data.__dict__['log']) # see 'Time-stepping time (s)'

[05:40:53] WARNING: The wave vector of source 'None' along dimension '0' is     
           equal to the Bloch vector of the simulation boundaries along that    
           dimension plus an integer reciprocal lattice vector. If using a      
           'DiffractionMonitor', diffraction order 0 will not correspond to the 
           angle of propagation of the source. Consider using                   
           'BlochBoundary.from_source()'.                                       
           equal to the Bloch vector of the simulation boundaries along that    
           dimension plus an integer reciprocal lattice vector. If using a      
           'DiffractionMonitor', diffraction order 0 will not correspond to the 
           angle of propagation of the source. Consider using                   
           'BlochBoundary.from_source()'.                                       
           equal to the Bloch vector of the simulation boundaries along that    
           dimension plus an